## $dY_{t} =  f(Y_{t}) \,dt + g(Y_{t})\, dW_{t}$

General Form of SDE

# 1 Dimensional Example


In [2]:
from SDEsim import SDE
#help(SDE)

### $dY_t =  -Y_t \,dt + 0.1\,dW_t$

In [44]:
#Declaring a simulation with 40,000,000 parallel solvers
sim = SDE(1_000_000, dtype = 'float64', method = 'milstein', device = 'cuda', tolerance=1e-4, dt_init=1e-14)

#Inputting simulation equation
f = lambda y: -y #drift
g = lambda y: 0.01 #diffusion
dg = lambda y: 0.0 #derivative of g, not required when method = 'euler_maruyama'
df = lambda y: -1.0 #derivative of drift, it is required for measuring information rate of velocity or if using methods with higher order than milstein
#sim.debug=False
sim.set_functions(f, g, df, dg) #if dg or df not given it will automatically be assumed to be 0.
#Setting up stats measurements, with collection_frequency, once every 2 time steps
sim.set_stats(info_rate=True, info_rate_v=False, density= False, density_v=False, mean_v=False, std_v=False, mean=True, std=True, entropy=False,entropy_v=False,collection_frequency=1)

In [45]:
#run simulation
sim.simulate(t_init=0, t_end=100, init_mean=10.0, init_std=0.00001)
#save data
sim.save_data(output_directory = r'data', filename = f'linear.dic')

Starting Simulation...
<==================================================> dt: 2.90e-02 acc: 3695 rej: 145 ETA: -0s                 
Simulation Finished: 6.24s


### $dY_t =  -\left(Y_t^3 - 2Y_t\right)\,dt+0.8\,dW_t$

In [ ]:
f = lambda y: -y**3 + 2*y #drift
g = lambda y: 0.8 #diffusion
dg = lambda y: 0.0 #derivative of g, not required when method = 'euler_maruyama'
df = lambda y: -3*y**2 + 2 #derivative of drift, it is required for measuring information rate of velocity or if using methods with higher order than milstein
sim.set_functions(f, g, df,dg)

sim.simulate(t_init=0, t_end=100, init_mean=0.0, init_std=0.5)
sim.save_data(output_directory = r'data', filename = f'non_linear.dic')

## Custom Measurement: Auto-Correlation Time

Auto-Correlation is defined as:
$
\mathcal{C}(t,t') := \langle \delta Y(t) \delta Y(t') \rangle
$
<br>
$
\delta Y(t) := Y(t) - \langle Y(t) \rangle
$
<br>
Normalized Auto-Correlation is defined as:
$
\mathcal{\tilde{C}}(t,t') := \frac{\langle \delta Y(t) \delta Y(t') \rangle}{\sqrt{\langle \delta Y(t)^2 \rangle \langle \delta Y(t')^2 \rangle}}
$
<br>
Correlation functions usually depends on time intervals.
$
\mathcal{C}(t,t') \equiv \mathcal{C}(t-t') 
$
<br>
$
\tau := t-t'
$

Correlation time is defined as:
$
\tau_{c}(t')=\int_{0}^{\infty} d t \mathcal{\tilde{C}}(t,t')
$

In [ ]:
import torch
import numpy as np

sim = SDE(1_000_000, dtype = 'float64', method = 'milstein', device = 'cuda', tolerance=1e-5, dt_init=1e-10)
f = lambda y: -y #drift
g = lambda y: 0.1 #diffusion
dg = lambda y: 0.0 #derivative of g, not required when method = 'euler_maruyama'
df = lambda y: -1.0 #derivative of drift, it is required for measuring information rate of velocity or if using methods with higher order than milstein

sim.set_functions(f, g, df, dg)
#sim.initialize(t_init=0, t_end=100, init_mean=100, init_std=0.0001)

In [16]:
t_prime = 1e-10

#If initial dt greater than t_prime, make it smaller.
if sim.dt_init>t_prime: 
    sim.dt_init *= 0 #to preserve the datastructure
    sim.dt_init += t_prime*0.1 #start with a smaller initial dt that required smalled t_prime value
    
sim.initialize(t_init=0, t_end=100, init_mean=1e5, init_std=0.00001)

#Get the value of y at t_prime
while(sim.t < t_prime):
    if sim.t+sim.dt > t_prime:
        sim.dt = t_prime - sim.t
    sim.step()
y_0 = sim.y

corr_list = []
t_list = []

while(normalized_corr>1e-4): #Truncate measurement of normalized correlation according to some criteria
    normalized_corr = (torch.mean(y_0*sim.y)-y_0.mean()*sim.y.mean())/(y_0.std()*sim.y.std())
    corr_list.append(normalized_corr)
    t_list.append(sim.t)
    sim.step()
    print(sim.t.item(), normalized_corr.item(), end='\r')

tau_c = torch.trapz(torch.stack(corr_list), torch.stack(t_list))

# N Dimensional Example

$
\partial_{t} \mathcal{E}=\mathcal{E} \mathcal{N}-a_{1} \mathcal{E}^{2}-a_{2} V^{2} \mathcal{E}-a_{3} V_{\mathrm{ZF}}^{2} \mathcal{E} 
\\
\partial_{t} V_{\mathrm{ZF}}=b_{1} \frac{\mathcal{E} V_{\mathrm{ZF}}}{1+b_{2} V^{2}}-b_{3} V_{\mathrm{ZF}} 
\\
\partial_{t} \mathcal{N}=-c_{1} \mathcal{E} \mathcal{N}-c_{2} \mathcal{N}+t_c \, t + 0.001\,dW_t
$

<br>
These equations are stiff. Numerical solution requires very small tolerance. This will cause significant slow down.
This method is only advantageus when running large number of parallel simulations (order of millions) with CUDA.

In [ ]:
from SDEsim import SDE_ND

In [ ]:
sim = SDE_ND(2_000_000, dims = 3, dtype='float64', device = 'cuda', method='milstein', dt_init=1e-10)

In [ ]:
sim.debug=True

sim.tolerance = 1e-9 #very low tolerance because of stiffness in the equation
sim.facmax = 1.05 #the adaptive step size will increase by a maximum factor of 1.05

a1, a2, a3 = 0.2, 0.7, 0.7
b1, b2, b3 = 1.5, 1, 1
c1, c2, d = 1, 0.5, 1
tc = 0.01

f0 = f'y0*y2 - {a1}*y0**2 - {a2*d**2}*y2**4*y0 - {a3}*y1**2*y0'
f1 = f'{b1}*y0*y1/(1+{b2*d**2}*y2**4) - {b3}*y1'
f2 = f'-{c1}*y0*y2 - {c2}*y2 + {tc}*t + 0.1'

g0 = 0
g1 = 0
g2 = lambda x: 1e-3

#dg (derivative of g) is 0 for all cases so not given
sim.set_functions((f0,g0),(f1,g1),(f2,g2))
#collection_frequency=10, collects stats every 10 time steps.
sim.set_stats(mean_v = True, std_v = True, mean=True, std=True, entropy=True, density=True, entropy_v=True, collection_frequency=10)


In [ ]:
sim.simulate(t_init=0, t_end=200, init_mean=0.01, init_std=0.0)
#sim.save_data(output_directory = r'data', filename = f'ND.dic')

### Plotting

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
x = sim.data['t'][::]*0.01
eqs = [r'$\mathcal{\epsilon}$', r'$V_{ZF}$', r'$\mathcal{N}/5$']

for i in range(3):
    y = sim.data['mean(t)'][::,i]
    y_std = sim.data['std(t)'][::,i]*10
    if i==2: y=y/5; y_std=y_std/5
        
    plt.plot(x, y, label = eqs[i])
    plt.fill_between(x, y-y_std, y+y_std, alpha=0.3)
    
#x = temp.data['t'][::]*0.01
eqs = ['1e-5::'+i for i in eqs]

plt.xlabel('Q')
plt.ylabel('y')
plt.legend()
#plt.yscale('log')
plt.show()

# Experiments
### Ignore this section.

$d X_{t}=\left[-1000 X_{t}\left(1-X_{t}\right)\left(2-X_{t}\right)\right] d t+10\, d W_{t}, \quad X_{0}=2, \quad t \in[0,5]$

In [37]:
#help(SDE)
#Declaring a simulation with 40,000,000 parallel solvers
sim = SDE(1, dtype = 'float64', method = 'milstein', device = 'cpu', tolerance=1e-6)
#Inputting simulation equation
f = lambda x: -1000*x*(1-x)*(2-x) #drift
g = lambda y: 10 #diffusion

#sim.debug=False
sim.set_functions(f, g) #if dg or df not given it will automatically be assumed to be 0.
#Setting up stats measurements, with collection_frequency, once every 2 time steps
sim.set_stats(info_rate=True, info_rate_v=False, density= True, density_v=False, mean_v=False, std_v=False, mean=True, std=True, entropy=False,entropy_v=False,collection_frequency=2)

In [38]:
sim.simulate(t_init=0, t_end=5, init_mean=3, init_std=0)

Starting Simulation...


KeyboardInterrupt: 